In [1]:
import tkinter
from tkinter import messagebox
import time
import numpy as np
from huobi.client.market import MarketClient
from huobi.constant import *
from huobi.utils import *
import pandas as pd
import matplotlib.pyplot as plt
from huobi.client.account import AccountClient
import pickle
from huobi.privateconfig import p_api_key, p_secret_key
import requests


In [2]:
def alert(msg:str):
    root = tkinter.Tk()
    root.withdraw()
    root.lift()
    messagebox.showinfo("Title", msg)

In [3]:
market_client = MarketClient(init_log=True)

In [4]:
interval = CandlestickInterval.MIN5
symbol = "eth3lusdt"
list_obj = market_client.get_candlestick(symbol, interval, 2)
LogInfo.output("---- {interval} candlestick for {symbol} ----".format(interval=interval, symbol=symbol))
LogInfo.output_list(list_obj)

---- 5min candlestick for eth3lusdt ----
Id : 1614141300
High : 47.1
Low : 46.6981
Open : 47.0169
Close : 46.8033
Count : 81
Amount : 386.78935212576704
Volume : 18141.01399317

Id : 1614141000
High : 47.6627
Low : 46.8062
Open : 47.0171
Close : 47.097
Count : 565
Amount : 2546.8441277798565
Volume : 120351.30612621



In [5]:
def get_etp_detail(name):
    url=f"https://api.huobi.be/market/etp?symbol={name}"
    respond = requests.get(url)
    amounts = respond.json()['tick']['basket']
    res = {i['currency']:i['amount'] for i in amounts}
    return res
res=get_etp_detail("btc3lusdt")

In [6]:
t=requests.get("https://api.huobi.be/market/etp?symbol=eth3lusdt")

In [7]:
t.json()

{'ch': 'market.eth3lusdt.etp',
 'status': 'ok',
 'ts': 1614141329707,
 'tick': {'actualLeverage': 2.841095123508893,
  'nav': 47.084515145528584,
  'outstanding': 778158.01217471,
  'symbol': 'eth3lusdt',
  'navTime': 1614141329659,
  'basket': [{'amount': 0.08229616077167001, 'currency': 'eth'},
   {'amount': -86.6870712272133, 'currency': 'usdt'}]}}

In [8]:
def listen(market_client,listen_prices,symbol="ethusdt"):
    i=0
    while True:
        i+=1
        interval = CandlestickInterval.MIN1
        list_obj = market_client.get_candlestick(symbol, interval, 1)
        cur=list_obj[0]
        for price in listen_prices:
            if cur.low<=price<=cur.high:
                alert(f"{symbol} reaches {price:.2f}")
                return
        if i%20==0:
            print(cur.low, cur.high)
        time.sleep(30)

In [9]:
# listen(market_client,[1535,1515])

In [42]:
# short
res=get_etp_detail("btc3susdt")
USDT=res['usdt']
COIN =-res['btc']
def target_price2(y):
    return (USDT-y)/COIN

In [43]:
x=np.arange(0.014,0.021,0.0001)
# y=[target_price(i) for i in x]
for i in x:
    print(f"{i:0.5f}", target_price2(i))

0.01400 17502.13346292315
0.01410 17135.700225312103
0.01420 16769.26698770106
0.01430 16402.833750090016
0.01440 16036.400512478971
0.01450 15669.967274867926
0.01460 15303.534037256883
0.01470 14937.100799645837
0.01480 14570.667562034792
0.01490 14204.234324423749
0.01500 13837.801086812704
0.01510 13471.36784920166
0.01520 13104.934611590616
0.01530 12738.501373979572
0.01540 12372.068136368527
0.01550 12005.634898757482
0.01560 11639.201661146439
0.01570 11272.768423535388
0.01580 10906.335185924356
0.01590 10539.9019483133
0.01600 10173.468710702267
0.01610 9807.03547309121
0.01620 9440.602235480179
0.01630 9074.168997869121
0.01640 8707.73576025809
0.01650 8341.302522647033
0.01660 7974.869285036001
0.01670 7608.436047424944
0.01680 7242.002809813913
0.01690 6875.569572202856
0.01700 6509.136334591824
0.01710 6142.7030969807665
0.01720 5776.269859369735
0.01730 5409.836621758678
0.01740 5043.403384147647
0.01750 4676.97014653659
0.01760 4310.536908925558
0.01770 3944.10367131450

In [7]:
# long
SYMBOL="eth"
res=get_etp_detail(f"{SYMBOL}3lusdt")
USDT=-res['usdt']
COIN=res[SYMBOL]
def target_price(y):
    return (y+USDT)/COIN

In [13]:
46.5*1.03

47.895

In [8]:
x=np.arange(50,100,1)
# y=[target_price(i) for i in x]
for i in x:
    print(f"{i:0.3f}", target_price(i))

50.000 1660.9167419905577
51.000 1673.067977122589
52.000 1685.2192122546198
53.000 1697.370447386651
54.000 1709.5216825186822
55.000 1721.6729176507133
56.000 1733.8241527827445
57.000 1745.9753879147756
58.000 1758.1266230468068
59.000 1770.277858178838
60.000 1782.4290933108691
61.000 1794.5803284429003
62.000 1806.7315635749314
63.000 1818.8827987069626
64.000 1831.0340338389938
65.000 1843.185268971025
66.000 1855.336504103056
67.000 1867.487739235087
68.000 1879.6389743671182
69.000 1891.7902094991493
70.000 1903.9414446311805
71.000 1916.0926797632117
72.000 1928.2439148952428
73.000 1940.395150027274
74.000 1952.5463851593051
75.000 1964.6976202913363
76.000 1976.8488554233675
77.000 1989.0000905553986
78.000 2001.1513256874298
79.000 2013.302560819461
80.000 2025.453795951492
81.000 2037.605031083523
82.000 2049.756266215554
83.000 2061.9075013475854
84.000 2074.0587364796165
85.000 2086.2099716116477
86.000 2098.361206743679
87.000 2110.51244187571
88.000 2122.663677007741
8

In [11]:
def app(x):
    print(x,x*1.01,x*1.03,x*1.05,x*1.1)
app(542)

542 547.42 558.26 569.1 596.2


In [9]:
from websocket import create_connection

ws = create_connection("wss://api.huobi.pro/ws", timeout=5)
if ws.connected:
      ws.send('8')
      print(ws.recv())

2021-02-08 15:44:27,461 - huobi-client - INFO - [Sub][1] Connecting...
2021-02-08 15:44:27,461 - huobi-client - INFO - [Sub][1] Connecting...
2021-02-08 15:44:27,461 - huobi-client - INFO - [Sub][1] Connecting...
2021-02-08 15:44:27,461 - huobi-client - INFO - [Sub][1] Connecting...
2021-02-08 15:44:48,524 - huobi-client - ERROR - [Sub][1] Unexpected error: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond
2021-02-08 15:44:48,524 - huobi-client - ERROR - [Sub][1] Unexpected error: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond
2021-02-08 15:44:48,524 - huobi-client - ERROR - [Sub][1] Unexpected error: [WinError 10060] A connection attempt failed because the connected party did not properl

In [40]:
import pickle

In [41]:
with open('save.pkl', 'rb') as f:
    t=pickle.load(f)

In [42]:
len(t)

54989

In [45]:
k=sorted(t, key=lambda x:x.id)

In [47]:
k[0].close

28822.69

In [5]:
11140/(60)

185.66666666666666

In [6]:
c=10**9+7
print(id(c))
c=10**9+8
print(id(c))

2185838296368
2185838293712


In [13]:
c=[]
print(id(c))
c.append(1)
print(id(c))

2185904332552
2185904332552


In [26]:
def func():
    c=0
    print(id(c))
    def inner():
        c=1 # re-assign to local variable
        print(id(c)) 
        return
    inner()
    return

In [27]:
func()

140731315954032
140731315954064


In [28]:
c=1
c=2 # spawn new variable c, old `c=1` shadowed

In [3]:
from huobi.client.market import MarketClient
from huobi.constant import *
from huobi.exception.huobi_api_exception import HuobiApiException
import pickle
import time

SYMBOL = "btc3susdt"

save=[]


def callback(candlestick_req: 'CandlestickReq'):
    save.extend(candlestick_req.data)

def error(e: 'HuobiApiException'):
    print(e.error_code + e.error_message)

In [74]:

sub_client = MarketClient(init_log=True)
for i in range(1609430400, 1612772463,300*60):
    sub_client.req_candlestick(SYMBOL, CandlestickInterval.MIN1, callback, from_ts_second=i, end_ts_second=i+299*60, error_handler=error)
    print(i)
    time.sleep(0.5)

246 - huobi-client - INFO - [Sub][240] Connected to server
2021-02-08 17:18:37,229 - huobi-client - INFO - [Sub][232] Connection event loop down
2021-02-08 17:18:37,246 - huobi-client - INFO - [Sub][240] Connected to server
2021-02-08 17:18:37,229 - huobi-client - INFO - [Sub][232] Connection event loop down
2021-02-08 17:18:37,246 - huobi-client - INFO - [Sub][240] Connected to server
2021-02-08 17:18:37,229 - huobi-client - INFO - [Sub][232] Connection event loop down
2021-02-08 17:18:37,246 - huobi-client - INFO - [Sub][240] Connected to server
2021-02-08 17:18:37,229 - huobi-client - INFO - [Sub][232] Connection event loop down
2021-02-08 17:18:37,246 - huobi-client - INFO - [Sub][240] Connected to server
2021-02-08 17:18:37,229 - huobi-client - INFO - [Sub][232] Connection event loop down
2021-02-08 17:18:37,246 - huobi-client - INFO - [Sub][240] Connected to server
2021-02-08 17:18:37,229 - huobi-client - INFO - [Sub][232] Connection event loop down
2021-02-08 17:18:37,246 - huob

In [29]:
market_client = MarketClient(init_log=True)
interval = CandlestickInterval.MIN1
symbol = "btc3lusdt"
list_obj = market_client.get_candlestick(symbol, interval, 1)
LogInfo.output("---- {interval} candlestick for {symbol} ----".format(interval=interval, symbol=symbol))
LogInfo.output_list(list_obj)

---- 1min candlestick for btc3lusdt ----
Id : 1613992380
High : 484.5215
Low : 482.2975
Open : 484.0001
Close : 482.2975
Count : 23
Amount : 22.210933405638347
Volume : 10731.44512335



In [32]:
Decimal(list_obj[0].close).quantize(Decimal('.0001'), rounding=ROUND_DOWN)

Decimal('482.2975')

In [2]:
from huobi.privateconfig import p_api_key, p_secret_key
from huobi.client.trade import TradeClient
from huobi.client.market import MarketClient
from huobi.client.wallet import WalletClient
from huobi.client.account import AccountClient
from huobi.constant import CandlestickInterval

In [3]:
import time
import datetime
from huobi.privateconfig import p_api_key, p_secret_key
from huobi.client.trade import TradeClient
from huobi.client.market import MarketClient
from huobi.client.wallet import WalletClient
from huobi.client.account import AccountClient
from huobi.constant import *
from decimal import *
import pickle
import os


class Trader:
    def __init__(self):
        self.cash = 0
        self.stock = 0
        self.history_data = []
        self.active_buy_orders = {}
        # Dict: order_id: order_id, client_order_id, order_price, time.time()
        self.client_order_id = 6
        self.config_file = "config.pkl"
        self.order_price = Decimal(0)
        self.target_symbol = "btc3l"
        self.pair_symbol = self.target_symbol + "usdt"
        self.trade_log_file = "trade_log.txt"
        self.run_log_file = "run_log.txt"
        self.error_log_file = "error_log.txt"
        self.account_id = 1037218
        self.last_buy_time = 0
        self.buy_cooling_time = 30  # second
        self.cur_time = 0
        self.trade_client = TradeClient(api_key=p_api_key, secret_key=p_secret_key, init_log=True)
        self.market_client = MarketClient()
        self.account_client = AccountClient(api_key=p_api_key, secret_key=p_secret_key)
        self.balances_detail = self.account_client.get_balance(self.account_id)

        if os.path.exists(self.config_file):
            with open(self.config_file, 'rb') as f:
                self.client_order_id, self.active_buy_orders = pickle.load(f)

    def run(self):
        while True:
            try:
                self.cur_time = int(time.time())
                if self.check_buy_condition():
                    self.buy()
            except Exception as e:
                self.error_log(str(e))
            time.sleep(2)
            try:
                self.update_orders()
            except Exception as e:
                self.error_log(str(e))
            time.sleep(3)

    def check_buy_condition(self) -> bool:
        if self.cur_time < self.last_buy_time + self.buy_cooling_time:
            self.trade_log(f"TRY BUYING but in COLDTIME at {self.order_price:0.4f}")
            return False
        interval = CandlestickInterval.MIN1
        length = 30
        list_obj = self.market_client.get_candlestick(self.pair_symbol, interval, length + 1)
        price_sum = 0

        for candlestick in list_obj[1:]:
            price_sum += candlestick.close
        price_sum_average = price_sum / length

        self.run_log(
            f"{self.client_order_id} {self.active_buy_orders} {list_obj[0].close:0.4f} " +
            f"{price_sum_average:0.4f} {price_sum_average*0.95:0.4f}")

        if list_obj[0].close < price_sum_average * 0.95:
            self.order_price = Decimal(list_obj[0].close).quantize(Decimal('.0001'), rounding=ROUND_DOWN)
            return True
        else:
            return False

    def buy(self):
        division = Decimal(10)
        self.update_balance()
        usdt_balance = Decimal(self.get_currency_balance("usdt"))
        min_order = Decimal(6)
        if usdt_balance < min_order:
            self.trade_log(f"TRY BUYING but NO ENOUGH MONEY at {self.order_price:0.4f}")
            return
        symbol_balance = self.get_total_currency_balance(self.target_symbol)
        symbol_price = self.get_lastest_price(self.pair_symbol)
        total_balance = usdt_balance + symbol_balance * symbol_price

        buy_amount_usdt = max(min_order, min(total_balance / division, usdt_balance))
        buy_amount = (buy_amount_usdt / self.order_price).quantize(Decimal('.0001'), rounding=ROUND_DOWN)

        cur_timestamp = self.cur_time
        self.client_order_id += 1
        try:
            order_id = self.trade_client.create_order(self.pair_symbol, self.account_id, OrderType.BUY_LIMIT,
                                                      buy_amount, self.order_price, source=OrderSource.API,
                                                      client_order_id=str(self.client_order_id))
            self.last_buy_time = cur_timestamp
            self.trade_log(f"{order_id}: BUY ORDERED.   BUY {buy_amount} at {self.order_price}")
            self.active_buy_orders[order_id] = [order_id, self.client_order_id, self.order_price, cur_timestamp]
        except Exception as e:
            self.trade_log(f"TRY BUYING but FAIL at {self.order_price:0.4f}")
            raise(e)

    def update_orders(self):
        cur_timestamp = self.cur_time
        waiting_limit = 60 * 5
        for order_id in list(self.active_buy_orders):
            _, _, order_price, order_cur_timestamp = self.active_buy_orders[order_id]
            order_obj = self.trade_client.get_order(order_id=order_id)
            order_state = order_obj.state
            if order_state == "filled":
                sell_amount = (Decimal(order_obj.filled_amount) - Decimal(order_obj.filled_fees)
                               ).quantize(Decimal('.0001'), rounding=ROUND_DOWN)
                sell_price = (order_price * Decimal(1.03)).quantize(Decimal('.0001'), rounding=ROUND_DOWN)

                self.trade_log(f"{order_id}: BUY SUCCEEDED. BUY {sell_amount} at {order_price:0.4f}")
                self.client_order_id += 1
                try:
                    sell_order_id = self.trade_client.create_order(
                        self.pair_symbol, self.account_id, OrderType.SELL_LIMIT,
                        sell_amount, sell_price, source=OrderSource.API,
                        client_order_id=str(self.client_order_id))

                    self.trade_log(f"{sell_order_id}: SELL ORDERED.  SELL {sell_amount} at {sell_price:0.4f}")
                    del self.active_buy_orders[order_id]
                except Exception as e:
                    self.trade_log(f"TRY SELLING but FAIL at ORDER: {order_id}")
                    raise(e)

            elif cur_timestamp - order_cur_timestamp > waiting_limit:
                self.trade_client.cancel_order(self.pair_symbol, order_id)
                self.trade_log(f"CANCEL BUYING at {self.order_price:0.4f}")
                del self.active_buy_orders[order_id]

        with open(self.config_file, 'wb') as f:
            pickle.dump((self.client_order_id, self.active_buy_orders), f)

    def get_currency_balance(self, currency: str) -> str:
        balances = self.balances_detail
        for balance in balances:
            if balance.currency == currency and balance.type == "trade":
                return balance.balance

    def get_total_currency_balance(self, currency: str) -> Decimal:
        balances = self.balances_detail
        total_balance = Decimal(0)
        for balance in balances:
            if balance.currency == currency:
                total_balance += Decimal(balance.balance)
        return total_balance

    def get_asset_valuation(self) -> str:
        asset_valuation = self.account_client.get_account_asset_valuation(
            account_type="spot", valuation_currency="usd")
        return asset_valuation.balance

    def get_lastest_price(self, pair_symbol) -> Decimal:
        return Decimal(self.market_client.get_market_trade(symbol=pair_symbol)[0].price)

    def update_balance(self):
        self.balances_detail = self.account_client.get_balance(self.account_id)

    def trade_log(self, msg: str):
        cur_time = datetime.datetime.fromtimestamp(self.cur_time).strftime(r"%Y%m%d_%H%M%S")
        print(cur_time, msg)
        with open(self.trade_log_file, "a+") as f:
            f.write(f"{cur_time} {msg}\n")

    def run_log(self, msg: str):
        cur_time = datetime.datetime.fromtimestamp(self.cur_time).strftime(r"%Y%m%d_%H%M%S")
        print(cur_time, msg)
        with open(self.run_log_file, "a+") as f:
            f.write(f"{cur_time} {msg}\n")

    def error_log(self, msg: str):
        cur_time = datetime.datetime.fromtimestamp(self.cur_time).strftime(r"%Y%m%d_%H%M%S")
        print(cur_time, msg)
        with open(self.error_log_file, "a+") as f:
            f.write(f"{cur_time} {msg}\n")



In [4]:
t=Trader(
)

In [5]:
t.order_price=10

In [6]:
t.buy()

19700101_080000 226008885863896: BUY ORDERED.   BUY 0.9643 at 10


In [7]:
division = Decimal(10)
t.update_balance()
usdt_balance = Decimal(t.get_currency_balance("usdt"))
min_order = Decimal(6)
symbol_balance = t.get_total_currency_balance(t.target_symbol)
symbol_price = t.get_lastest_price(t.pair_symbol)
total_balance = usdt_balance + symbol_balance * symbol_price

In [11]:
t.get_total_currency_balance("usdt")

Decimal('95.584093721069477107')

In [55]:
usdt_balance = Decimal(t.get_currency_balance("usdt"))
btc3l_balance = t.get_total_currency_balance("btc3l")
btc3l_price = t.get_lastest_price("btc3lusdt")
total_balance = usdt_balance + btc3l_balance * btc3l_price

In [56]:
total_balance

Decimal('77.88349934028738938311726427')

In [46]:
t.get_currency_balance("btc3l")

Decimal('0.052077201436509253')

In [27]:
from huobi.client.trade import TradeClient
from huobi.constant import *
from huobi.utils import *

symbol = "btc3lusdt"
list_obj = t.trade_client.get_orders(symbol=symbol, order_state=OrderState.SUBMITTED)


In [28]:
Decimal(list_obj[0].close)

IndexError: list index out of range

In [13]:
t.check_buy_condition()

False

In [147]:
t.active_buy_orders

{}

In [40]:
def check():
    while True:
        if t.check_buy_condition():
            alert("buy!!")
        time.sleep(2)

In [ ]:
check()

In [13]:
balances = t.account_client.get_balance(1037218)

In [21]:
for balance in balances:
    if balance.currency == "btc3l":
        print(balance.balance)

0.060232213775299004
0


In [89]:
a=MarketClient()

In [90]:
kk=a.get_candlestick("btc3lusdt", CandlestickInterval.MIN1, 30)

In [92]:
type(kk[0].close)

float

In [41]:
t.account_id

1037218

In [ ]:
204610773149465

In [35]:
id(1)

140727743193488

In [34]:
dir([])

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [24]:
487.6239*0.996

485.6734044

In [45]:
import datetime
datetime.datetime.fromtimestamp(time.time()).strftime(r"%Y%m%d_%H%M%S")

'20210222_192457'

In [43]:
time.strftime(r"%Y%m%d_%H%M%S",time.time())

SyntaxError: positional argument follows keyword argument (<ipython-input-43-a3fbe5f5c34f>, line 1)

In [25]:
t.trade_client.create_order("btc3lusdt", t.account_id, OrderType.BUY_LIMIT,1, Decimal("asdasdadas"), source=OrderSource.API,
client_order_id="1000000000107")

HuobiApiException: ('ExecuteError', '[Executing] order-orderprice-precision-error: order price precision error, scale: `4`')

In [12]:
t.trade_client.cancel_order("btc3lusdt", 226008885863896)

226008885863896

In [28]:
orderObj = t.trade_client.get_order(order_id=204610773149465)

In [81]:
dir(orderObj)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'account_id',
 'amount',
 'canceled_at',
 'client_order_id',
 'created_at',
 'filled_amount',
 'filled_cash_amount',
 'filled_fees',
 'finished_at',
 'id',
 'json_parse',
 'json_parse_list',
 'next_time',
 'operator',
 'price',
 'print_object',
 'source',
 'state',
 'stop_price',
 'symbol',
 'type']

In [30]:
orderObj.state

'submitted'

In [74]:
(float(orderObj.filled_amount) - float(orderObj.filled_fees))*10000

0.998

In [93]:
type(time.time())

float

In [104]:
import math

In [94]:
from decimal import *

In [106]:
math.floor(Decimal('10000.000')/Decimal('6'),2)

TypeError: floor() takes exactly one argument (2 given)

In [114]:
str((Decimal('10000.000')/Decimal(6.001)).quantize(Decimal('.01'), rounding=ROUND_DOWN))

'1666.38'

In [116]:
max(5,Decimal(4))


5

In [117]:
f"{Decimal(4)}aaa"

'4aaa'

In [118]:
t.account_client.get_account_asset_valuation(
            account_type="spot", valuation_currency="usd")

In [119]:
a=_

In [120]:
a.balance

'47.11'

In [3]:
a=1
b=2

In [4]:
with open("zzz.pkl", 'wb') as f:
    pickle.dump((a,b), f)

In [5]:
with open("zzz.pkl", 'rb') as f:
    a,b=pickle.load(f)

In [7]:
b

2